In [ ]:
# Numpy helps with linear algebra operations.
import numpy 
# Scipy.special for the sigmoid function expit().
import scipy.special
# Library for plotting arrays.
import matplotlib.pyplot

In [ ]:
# Neural network class definition.
class neuralNetwork:
    
    # Initialise the neural network. Equivalent to constructor in C#.
    def __init__(self, inputNodes, hiddenNodes, outputNodes, learningrate):
        # Set number of nodes in each layer.
        self.inputNodes = inputNodes
        self.hiddenNodes = hiddenNodes
        self.outputNodes = outputNodes
        
        # Link-weight matrices, weigths_InputToHidden and weigths_HiddenToOutput.
        # Weights inside the arrays are w_i_j (Weight sub i sub j), where link is 
        # from node i in current layer to node j in the next layer.
        self.weigths_InputToHidden = numpy.random.normal(0.0, pow(self.inputNodes, -0.5), (self.hiddenNodes, self.inputNodes))
        self.weigths_HiddenToOutput = numpy.random.normal(0.0, pow(self.hiddenNodes, -0.5), (self.outputNodes, self.hiddenNodes))

        # Learning rate modulates learning speed when optimizing and adjusting
        # weight matrices.
        self.learningRate = learningrate
        
        # The activation function is a lambda expretion containing the
        # sigmoid function, scipy.special.expit(x).
        self.activation_function = lambda x: scipy.special.expit(x)
        

    # Method to train the NeuralNetwork.
    def train(self, inputList, targets_list):
        # Convert inputs list to 2-dimensinal array.
        inputs = numpy.array(inputList, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        
        # Calculate signals into hidden layer, which are the inputs, multiplied
        # by the weight matrix from the input to the hidden layer. 
        inputToHiddenLayer = numpy.dot(self.weigths_InputToHidden, inputs)
        # Calculate signals emerging from hidden layer.
        outputFromHiddenLayer = self.activation_function(inputToHiddenLayer)
        
        # Calculate signals into final output layer, which are the outputs from 
        # the hidden layer, multiplied by the weight matrix from the hidden to
        # the final output layer.
        inputToFinalLayer = numpy.dot(self.weigths_HiddenToOutput, outputFromHiddenLayer)
        # Calculate the signals emerging from final output layer.
        outputFromFinalLayer = self.activation_function(inputToFinalLayer)
        
        # Output layer error is the (target - actual calculated value).
        errorsInFinalLayer = targets - outputFromFinalLayer
        # Hidden layer error is the errorsInFinalLayer, split by weights, 
        # then, recombined at hidden nodes.
        errorsInHiddenLayer = numpy.dot(self.weigths_HiddenToOutput.T, errorsInFinalLayer) 
        
        # Update the weights for the links between the hidden and output layers.
        self.weigths_HiddenToOutput += self.learningRate * numpy.dot((errorsInFinalLayer * outputFromFinalLayer * (1.0 - outputFromFinalLayer)), numpy.transpose(outputFromHiddenLayer))
        
        # Update the weights for the links between the input and hidden layers.
        self.weigths_InputToHidden += self.learningRate * numpy.dot((errorsInHiddenLayer * outputFromHiddenLayer * (1.0 - outputFromHiddenLayer)), numpy.transpose(inputs))
        

    
    # Query the neural network, once it has been trained.
    def query(self, inputList):
        # Convert inputs list to 2-dimensional array.
        inputs = numpy.array(inputList, ndmin=2).T
        
        # Calculate signals into hidden layer.
        inputToHiddenLayer = numpy.dot(self.weigths_InputToHidden, inputs)
        # Calculate the signals emerging from hidden layer.
        outputFromHiddenLayer = self.activation_function(inputToHiddenLayer)
        
        # Calculate signals into final output layer.
        inputToFinalLayer = numpy.dot(self.weigths_HiddenToOutput, outputFromHiddenLayer)
        # Calculate the signals emerging from final output layer.
        outputFromFinalLayer = self.activation_function(inputToFinalLayer)
        
        return outputFromFinalLayer

In [ ]:
# 784 input nodes, one for each pixel in a digit.
input_nodes = 784
# 300 nodes, 
hidden_nodes = 200
# 10 nodes, one for each number from 0 to 9.
output_nodes = 10

# Learning rate, moderates learning, so no neuron over-fits to no sample.
learning_rate = 0.1

nn = neuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)


In [ ]:
# load the mnist training data CSV file into a list
training_data_file = open("mnist_train_100.csv", 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

In [ ]:
for record in training_data_list:
        all_values = record.split(',')
        # Scale and shift the inputs for them to be in a range of (0, 1). 
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # Create the target output values.
        targets = numpy.zeros(output_nodes) + 0.01
        # all_values[0] is the target label for this record, 
        # meaning, the right value.
        targets[int(all_values[0])] = 0.99
        nn.train(inputs, targets)

In [ ]:
# Load testing data CSV file into a list.
test_data_file = open("mnist_test_10.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

In [ ]:
all_values = test_data_list[0].split(',')
# Reshape array of 784 as a matrix of 28 x 28.
image_array = numpy.asfarray(all_values[1:]).reshape((28,28))
# Display this digit graphically. 
matplotlib.pyplot.imshow(image_array, cmap = 'Greys', interpolation = 'None')

In [ ]:
nn.query((numpy.asfarray(all_values[1:]) / 255 * 0.99) + 0.01)